# Import Packages and Load Data

In [105]:
import numpy as np
import time
import math

In [106]:
size=1000

Note in the code below that the data are represented as 64-bit floating point numbers, which is the default.  As the name implies, this tack requires 64 bits of memory for each floating-point value and each bit needs to be considered when multiplication is conducted.

In [111]:
p = np.load(f'data/mnist1_{size}.npy').astype(np.float32)
q = np.load(f'data/mnist2_{size}.npy').astype(np.float32)
assert p.shape[0] == q.shape[0]
assert p.shape[1] == 784
assert q.shape[1] == 784

n = p.shape[0]
pixels = p.shape[1]
n,pixels,p.dtype

(1000, 784, dtype('float32'))

# Methods for Computing Distances

The methods shown below demonstrate a range of approaches for computing the pairwise distances between MNIST images in two data sets.  The available data sets contain either 10, 100, 1000, 10000, or 60000 images, each of which is constituted by 784 pixels encoded as floating-point values.  A data set with <code>n</code> images constitutes a <code>numpy</code> that is $n \times 784$.

## Base Python: Nested <code>for</code> Loops

In [110]:
start = time.time()
result = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        sum_sq = 0.0
        for k in range(pixels):
            sum_sq += (q[i][k] - p[j][k])**2
        result[i][j] = math.sqrt(sum_sq)

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

KeyboardInterrupt: 

## Expand Squared Difference with Matrix-Vector Math

In [112]:
start = time.time()
result = np.zeros((n,n)).astype(np.float32)
for i in range(n):
    for j in range(n):
        result[i][j] = np.sqrt(q[i]@q[i]-2*p[j]@q[i]+p[j]@p[j])

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

Exec. time: 7.344522476196289 for 1000x1000
[10.553867 10.057242  8.83383   9.928222  8.737532]


## Element-wise (Vectorized) Math with <code>numpy</code> and Nested <code>for</code> Loops

In [113]:
start = time.time()
result = np.zeros((n,n)).astype(np.float32)
for i in range(n):
    for j in range(n):
        result[i][j] = np.sqrt(((q[i]-p[j])**2).sum())

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

Exec. time: 4.896218776702881 for 1000x1000
[10.553867 10.057242  8.83383   9.928222  8.737532]


## <code>numpy np.newaxis()</code> with Broadcasting to Avoid Loops 

In [114]:
start = time.time()
#result = np.sqrt((q-p)@(q-p).T)
result = np.sqrt(((q[:, np.newaxis, :] - p)*(q[:, np.newaxis, :] - p)).sum(axis=2))

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

Exec. time: 3.5109074115753174 for 1000x1000
[10.553867 10.057242  8.83383   9.928222  8.737532]


## Using the <code>numpy</code> Method <code> np.linalg.norm()</code>

In [116]:
start = time.time()
#result = np.sqrt((q-p)@(q-p).T)
result = np.linalg.norm(q[:, np.newaxis, :] - p, axis=2)

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

Exec. time: 2.414586305618286 for 1000x1000
[10.553867 10.057242  8.83383   9.928222  8.737532]


# Using the <code>scipy</code> Method <code> scipy.spatial.distance.cdist()</code>

In [117]:
from scipy.spatial.distance import cdist
start = time.time()
result = cdist(q,p)
print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result[0,:5])

Exec. time: 0.46059441566467285 for 1000x1000
[10.5538673  10.05724184  8.83382935  9.92822226  8.73753215]


# Introduction to <code>np.einsum()</code>

This <code>numpy</code> quickly multiplies arrays while summing across specified axis.  It requires as input a string in "Einstein Notation" that determines how the arrays will be multipled (e.g., vector-matrix multiplcation versus elementwise (vectorized) multiplication).

## Einstein Notation

Einstein notation give a roadmap for how <code>numpy</code> arrays and vectors are to be multiplied using single-characters each of which corresponds to a dimension (think rows and columns) of a <code>numpy</code> that is to be multiplied.  Further, when <code>np.einsum()</code> is used sums are often computed on the array that results from the specified multiplication operation.  

A typical example of Einstein notation for multiplying two 2D arrays is:

``` python
'ij,jk->ik'
```

where

 - <code>i</code> represents the rows of the first input and <code>j</code> its column indices
 - <code>j</code> represents the rows of the second input and <code>k</code> its column indices
 - The commonality of <code>j</code> indicates that the first array has the same number of columns as the second array has rows.  (Essential for matrix math.)
 - The symbol <code>-></code> separates inputs on the left from the output on the right
 - <code>ik</code> indicates that the output array has rows corresponding with the first input and columns corresponding with the second input and, further, that the elements multiplied along <code>j</code> are to be summed.

As is demonstrated below, these two characteristics of the letters/indices used are important:

- The order of the indices in the Einstein notation
- Which indices used to represent original inputs are included in the specification of the output, or not
- Where input index parameters are not represented in the output, it indicates where summing is to take place.

The following page documents Einstien notation and gives examples: [link_fort_Einstein_Notation](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html)

Let's demonstrate with two small arrays.

In [119]:
mat1 = np.random.randint(0,5,(2,2))
mat2 = np.random.randint(0,5,(2,2))
print(mat1,'\n\n',mat2)

[[3 1]
 [0 4]] 

 [[2 3]
 [4 0]]


This first example does not use <code>np.einsum()</code> but it, rather, shows three calculations that we might make with the two arrays, respectively, 

- The dot product of the two arrays 
- Elementwise multiplication of the two arrays
- The dot product of the two arays, subsequently summed along axis 0
- The dot product of the two arays, subsequently summed along axis 1
- The dot product of the two arays, subsequently summed along both axes

In [120]:
''' Dot products/matrix-vector math mode '''
print('Dot Product\n',mat1@mat2)
print('\nDot Product, summed on axis=0\n',(mat1@mat2).sum(axis=0))
print('\nDot Product, summed on axis=1\n',(mat1@mat2).sum(axis=1))
print('\nDot Product, summed on both axes\n',(mat1@mat2).sum())

''' Elementwise/vectorized computations '''
print('\nElementwise Product\n', mat1*mat2)
print('\nElementwise Product, summed on axis=0\n',(mat1*mat2).sum(axis=0))
print('\nElementwise Product, summed on axis=1\n',(mat1*mat2).sum(axis=1))
print('\nElementwise Product, summed on both axes\n',(mat1*mat2).sum())

Dot Product
 [[10  9]
 [16  0]]

Dot Product, summed on axis=0
 [26  9]

Dot Product, summed on axis=1
 [19 16]

Dot Product, summed on both axes
 35

Elementwise Product
 [[6 3]
 [0 0]]

Elementwise Product, summed on axis=0
 [6 3]

Elementwise Product, summed on axis=1
 [9 0]

Elementwise Product, summed on both axes
 9


This first example of <code>np.einsum()</code> uses Einstein notation that denotes the dot product:

``` python
'ij,jk->ik'
```

Notably, the <code>j</code> relates to the column indices of the first input as well as the row indices of the second input, which indicates that the elements in the rows of the first input are to be multiplied by the columns in the second input, as is done with the dot product.  The <code>ik</code> indicates that the result has two dimensions with the row indices relating to the first input and the column indices relating to the second input, again, as is the case with the dot product.

Note that the absence of an output specification results in a dot product with the input specifications as shown here.

In [121]:
result = np.einsum('ij,jk->ik', mat1, mat2)
print(result)

[[10  9]
 [16  0]]


In [122]:
result = np.einsum('ij,jk', mat1, mat2)
print(result)

[[10  9]
 [16  0]]


It was claimed earlier that the order of the indices mattered, as is demonstrated below.  reversing the <code>i</code> and <code>k</code> in the output specification causes the output to be the transpose of the prior respective result.

In [123]:
result = np.einsum('ij,jk->ki', mat1, mat2)
print(result)

[[10 16]
 [ 9  0]]


The two computations below sum along either of the axes depending whether the output is speciified to be related with the <code>i</code> or <code>j</code> indices.  <code>i</code> refers to the rows of the first input, and so the sum is along the columns.  Conversely, specifying the output with <code>k</code> sums along axis 0.

In [124]:
result = np.einsum('ij,jk->k', mat1, mat2)
print(result)

[26  9]


In [125]:
result = np.einsum('ij,jk->i', mat1, mat2)
print(result)

[19 16]


Summing along both axes with dot product being indicated with the inputs requires the <code>-></code> symbol with no output specification. 

In [126]:
result = np.einsum('ij,jk->', mat1, mat2)
print(result)

35


Elementwise multiplication, without summing, is down by listing the indices characters in the same order, with both indices characters also listed in the output specification.  

In [127]:
result = np.einsum('ij,ij->ij', mat1, mat2)
print(result)

[[6 3]
 [0 0]]


Similarly to a previous example, one can generate the transpose result by switching the characters.

In [ ]:
result = np.einsum('ij,ij->ji', mat1, mat2)
print(result)

Summing along axes is done as shown before with dot products.  Listing <code>i</code> will cause the summation to occur across the unlisted indices (<code>j</code>), that is across the columns as in <code>axis=1</code>.  Conversely listing <code>j</code> in the output specification causes summation as in <code>axis=1</code>.

In [ ]:
result = np.einsum('ij,ij->i', mat1, mat2)
print(result)

In [ ]:
result = np.einsum('ij,ij->j', mat1, mat2)
print(result)

Summing along all axes in accomplishing by omitting an output specification.

In [ ]:
result = np.einsum('ij,ij', mat1, mat2)
print(result)

Einstein notation is just as applicable with one dimensional vectors (single dimension arrays) but they are, of course, represented with only one index character.

In [ ]:
vec_one = np.ones(mat1.shape[0])

In [ ]:
mat1

In [ ]:
print(np.einsum('ij,j',mat1,vec_one))

In [ ]:
print(np.einsum('ij,i',mat1,vec_one))

In [ ]:
print(np.einsum('ij,i->j',mat1,vec_one))

In [ ]:
print(np.einsum('i,ij',vec_one,mat1))

In [ ]:
print(np.einsum('ij,i->',mat1,vec_one))

# Computing Distances with <code>np.einsum()</code>

In [ ]:
start = time.time()
# for i in range(n):
#    for j in range(n):
#        result[i][j] = np.sqrt(q[i]@q[i]-2*p[j]@q[i]+p[j]@p[j])

result_ein = np.sqrt(np.einsum('ij,ij->i',q,q)[:,np.newaxis] - 2*q@p.T + np.einsum('ij,ij->i',p,p))

print('Exec. time: %s for %sx%s' % (str(float(time.time() - start)), str(n), str(n)))
print(result_ein[0,:5])

In [ ]:
np.absolute(result - result_ein).max()